## Imports


In [42]:
import os
from supabase import create_client, Client
from pathlib import Path
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd

## Get json from Supabase


In [20]:
# Get keys and url for supabase
url: str = os.environ.get("VITE_SUPABASE_URL")
key: str = os.environ.get("VITE_SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [24]:
response = supabase.table("users").select("*").execute()

APIResponse[~_ReturnT](data=[{'id': 'cbcbc11d-a455-443d-8fc2-f4645be44460', 'user': {'seed': 'Over the past several years, advanced footwear has become an essential tool for runners to improve their fitness and performance. However, they have continued facing challenges with heat building up inside running shoes that can lead to sweating and blisters. This will not only produce more discomfort, but also prevent them from increasing speed and endurance due to lingering pain. Therefore, future innovations need to overcome these issues with advanced footwear to improve the lives of runners across the globe.', 'steps': {'problem or task representation': 'present a task to be engaged in or a problem to be solved. Pose this yourself', 'preparation': 'build up or reactivate a store of information relevant to the problem or task, including a knowledge of response algorithms for working problems in the domain in question.', 'generation': 'generate response possibilities by searching through the

### Sort responses based on last push to database


In [29]:
responses = sorted(response.data, key=lambda x: x['timestamp'])
responses

[{'id': 'cbcbc11d-a455-443d-8fc2-f4645be44460',
  'user': {'seed': 'Over the past several years, advanced footwear has become an essential tool for runners to improve their fitness and performance. However, they have continued facing challenges with heat building up inside running shoes that can lead to sweating and blisters. This will not only produce more discomfort, but also prevent them from increasing speed and endurance due to lingering pain. Therefore, future innovations need to overcome these issues with advanced footwear to improve the lives of runners across the globe.',
   'steps': {'problem or task representation': 'present a task to be engaged in or a problem to be solved. Pose this yourself',
    'preparation': 'build up or reactivate a store of information relevant to the problem or task, including a knowledge of response algorithms for working problems in the domain in question.',
    'generation': 'generate response possibilities by searching through the available path

In [37]:
responses[0]

crit_label = "useful"
crit_def = 'degree to which something is valuable, helpful, or solves a problem.'


for k, v in responses[0]['user']['steps'].items():

    EVAL_STRING = prompt = (
        f"Based on the definition of '{crit_label}' — {crit_def} — "
        f"rate the following {k} ({simulation[i][k]}) "
        f"Use highly critical judgement and the entire range of this scale: "
        f"1 (very low {crit_label}), "
        f"2 (low {crit_label}), "
        f"3 (slightly low {crit_label}), "
        f"4 (moderate {crit_label}), "
        f"5 (slightly high {crit_label}), "
        f"6 (high {crit_label}), "
        f"7 (very high {crit_label}). "
        f"Respond with ONLY a single number for an overall rating and absolutely no additional text or explanation."
    )

problem or task representation present a task to be engaged in or a problem to be solved. Pose this yourself
preparation build up or reactivate a store of information relevant to the problem or task, including a knowledge of response algorithms for working problems in the domain in question.
generation generate response possibilities by searching through the available pathways and exploring features of the environment that are relevant to the task at hand.
validation 
test response possibility for correctness or appropriateness against factual knowledge and relevant criteria.
outcome assess if the test has been passed perfectly—if there is complete attainment of the original goal—the process terminates. If there is complete failure—if no reasonable response possibility has been generated—the process will also terminate. If there is some progress toward the goal—if at least a reasonable response possibility has been generated or if there is some evidence of "getting warmer"—the process 

In [51]:
# Loads key from .env, make sure key is named <GEMINI_KEY> in .env
load_dotenv()
load_dotenv(verbose=True)
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)
key = os.environ.get('GEMINI_KEY')
genai.configure(api_key=key)
model = genai.GenerativeModel("gemini-2.0-flash-exp")

In [ ]:
response = model.generate_content(
    "Explain how AI works, limit response to 1 sentence and a max of 100 characters")
print(response.text)

- use seed for first step
- for steps 2 - n, use previous llm response as context
- pass instructions for current step


In [93]:
seed = responses[0]['user']['seed']
cols = list(responses[0]['user']['steps'].keys())
cols.insert(0, "seed")
df = pd.DataFrame(columns=cols)
for i in range(responses[0]['user']['iters']):
    new_row = pd.DataFrame(
        [{'seed': seed, 'problem or task representation': seed}])
    df = pd.concat([df, new_row], ignore_index=True)
df.head(1)

,seed,problem or task representation,preparation,generation,validation,outcome
0,"Over the past several years, advanced footwear...","Over the past several years, advanced footwear...",NaN,NaN,NaN,NaN


In [95]:
df.shape

(10, 6)

In [ ]:
for row in range(df.shape[0]):
    for col in range(df.shape[1]):
        if col == 0 or col == 1:
            continue
        label = responses[0]['user']['steps'][df.columns[col]]
        prompt = (
            f"Given information about the following {str.upper(df.iloc[row, col-1])}"
            f"Step {str.upper(df.columns[col])}: {label} Please respond with ONLY the {df.columns[col]} step and absolutely no additional text or explanation."
        )
        genai.configure(api_key=key)
        model = genai.GenerativeModel("gemini-2.0-flash-exp")
        response = model.generate_content(prompt,
                                          generation_config=genai.types.GenerationConfig(
                                              temperature=1.0,))
        df.iloc[row, col] = response.text
    if row > 1:
        break

In [110]:
pd.set_option('display.max_colwidth', None)
df.to_csv('sample.csv', index=False)